In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [4]:
import ast

In [5]:
movies=pd.read_csv('tmdb_5000_movies.csv')

In [6]:
credits=pd.read_csv('tmdb_5000_credits.csv')

In [7]:
movies.head(1)

In [8]:
credits.head(1)['cast'].values

In [9]:
movies = movies.merge(credits,on='title')

In [10]:
movies.head(1)

In [11]:
#content based recommender system
#curse of dimension(eliminate unnecessary)
#genres
#id
#keywords
#title
#overview
#cast(heroine)
#crew(director)

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

In [12]:
movies.head(1)

In [13]:
#tags-overview+genres+keywords+cast(3mem)+crew(director)
#preprocessing and merging

movies.isnull().sum()

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.isnull().sum()

In [16]:
movies.duplicated().sum()

In [17]:
movies.iloc[0].genres

In [18]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action','Advanture','fantasy','Scifi']

In [19]:
#helper function
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [20]:
movies['genres'] = movies['genres'].apply(convert)

In [21]:
movies.head(1)

In [22]:
movies['keywords'] = movies['keywords'].apply(convert)

In [23]:
def convert3(obj):
    L = []
    counter=0
    for i in ast.literal_eval(obj):
        if counter !=3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [24]:
movies['cast'] = movies['cast'].apply(convert3)

In [25]:
movies.head()

In [26]:
movies['crew'][0]

In [27]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [28]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [29]:
movies.head()

In [30]:
movies['overview'][0]

In [31]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [32]:
movies.head()

In [33]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [34]:
#Sam Worthington->SamWorthington (differentiate)
movies['genres'].apply(lambda x:[i.replace(" ","")for i in x])
movies['keywords'].apply(lambda x:[i.replace(" ","")for i in x])
movies['cast'].apply(lambda x:[i.replace(" ","")for i in x])
movies['crew'].apply(lambda x:[i.replace(" ","")for i in x])

In [35]:
movies.head()

In [36]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']

In [37]:
movies.head()

In [38]:
new_df=movies[['movie_id','title','tags']]

In [39]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [40]:
new_df.head()

In [41]:
new_df['tags'][0]

In [42]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [43]:
new_df.head()

In [44]:
import nltk

In [45]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [46]:
def stem(text):
    y = []
    for i in text.split():
       y.append( ps.stem(i))
    
    return " ".join(y)

In [47]:
new_df['tags'] = new_df['tags'].apply(stem)

In [48]:
#similarities showing 
new_df['tags'][0]

In [49]:
new_df['tags'][1]

In [50]:
#vectorization(tag) closest vector ->recommendate same 
#bag of words tag-->vectors conversion(calculating frequency of words we were not considering stopwords)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [51]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [52]:
vectors

In [53]:
vectors[0]

In [54]:
cv.get_feature_names_out()

In [55]:
#stemming
ps.stem('dancing')

In [56]:
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction Sam Worthington Zoe Saldana Sigourney Weaver James Cameron'
)

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
similarity = cosine_similarity(vectors)

In [59]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [60]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
       
                         

In [61]:
recommend('Batman Begins')

In [62]:
new_df.iloc[1216].title

In [63]:
import pickle 

In [64]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [65]:
new_df['title'].values

In [68]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [69]:
pickle.dump(new_df,open('movie_list.pkl','wb'))

In [71]:
pickle.dump(similarity,open('similarity.pkl','wb'))